Partie réalisée par
- Calvet Rodolphe

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
from datetime import time
import networkx as nx
from networkx import *
from networkx import draw
import matplotlib.pyplot as plt
import seaborn as sns

import geocoder
import pickle
from ipyleaflet import Map, Marker, MarkerCluster, basemaps, FullScreenControl, AntPath, Polyline
from Algo_projet import *

# Sujet 2 : Etude des vélos et stations particulièrement utilisés

In [2]:
df= Prod_Clean_Df()
df.head(1)

load from disque OK


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,74203872,120,13382,2018-03-29 11:35:00,754,"Grenfell Road, Avondale",2018-03-29 11:33:00,442,"Walmer Road, Avondale"


On crée deux dataframes concernant le week-end : 
- l'un complet sur le samedi, pour étude des vélos les plus utilisés,
- l'autre complet sur le dimanche, pour les vélos moins utilisés,

et un dataframe concernant la semaine, entre 7 et 9h (horaire de bureau).

Ci dessous le code nous permet de jouer sur 3 paramètres :
- filtrer le master dataframe en samedi, dimanche ou semaine,
- filtrer en tranche horaire,
- filtrer en nombre de vélos à afficher SUR LA MAP, pour éventuellement améliorer la lisibilité, aussi bien sur les vélos les plus utilisés (en vert sur la map, avec un value_count et prise des plus utilisés, ou en rouge pour les moins utilisés avec un value_count/asc=True).

Nous avons naturellement fait varier ces paramètres au delà de ce qui est présenté ici, dans le but de répondre aux deux questions suivantes :

- Question 1 : peut t'on trouver une explication ou mieux comprendre le fait que, comme vu en partie 1, une fraction du parc est sur-utilisée au vu de la semaine entière,

- Question 2 : peut t'on observer une localité dans cet usage plus intense : ces vélos sont t'ils "prisonniers" temporairement d'une zone où la demande est plus/moins forte, y resteraient t'ils aléatoirement un certain temps avant d'en sortir?



## A - Etude sur l'utilisation des vélos durant le weekend (Question1)

In [3]:
#CREATION DES DATAFRAMES : param :  saturday ou sunday, Duration

# le samedi en entier : vélos les plus utilisés EN VERT sur la map
dF_We_sat =Get_Slice(df,"sat", (0,0), (23,30))

# le dimanche en entier :vélos les moins utilisés EN ROUGE sur la map
dF_We_sun = Get_Slice(df, "sun", (0,0), (23,30))

# CREATION DES GRAPHES : param : nbre de vélos étudiés, ici 10
G_sat = makeGraphTopBikes(dF_We_sat, 10)
G_sun = makeGraphLowBikes(dF_We_sun, 10)

# CREATION DES MARKERS (Pas forcément affichés pour lisibilité)
df_Coord_Gps_sat = Prod_Coord_Df(dF_We_sat)
df_Coord_Gps_sun = Prod_Coord_Df(dF_We_sun)

Je télécharge les Coord ETA 10 minutes
51.51367/-0.09161/Cheapside, Bank
51.51344145/-0.2344657143/BBC White City, White City
51.48063259/-0.2108041062/Aintree Street, Fulham
51.49015505/-0.02229992767/Spindrift Avenue, Millwall
51.49711121/-0.1897551234/St. Mark's Road, North Kensington
51.45436711/-0.1949528628/Neville Gill Close, Wandsworth
51.5370648/-0.07028651139/Belford House, Haggerston
51.47357392/-0.1770474955/Vicarage Crescent, Battersea
51.519264/-0.0731100593/Fournier Street, Whitechapel
51.52779/-0.02088996583/Bow Church Station, Bow
51.51500784/-0.02386068945/Stainsby Road , Poplar
51.51286504/-0.1581049059/Park Lane, Mayfair
51.5030911/-0.2155693554/Hansard Mews, Holland Park
51.53729223/-0.1427955949/Arlington Road, Camden Town
51.5464551/-0.1688201645/Lancaster Drive, Blackwall
51.50443/-0.15215/Park Lane , Hyde Park
51.59325021/-0.1638479394/Hertford Road, De Beauvoir Town
51.49393778/-0.1291932323/Page Street, Westminster
51.49247998/-0.1256800164/Millbank Tower, Pi

KeyboardInterrupt: 

Voyons l'ID des 10 vélos les plus utilisés le samedi et les moins utilisés le dimanche :

In [ ]:
sWEsat = dF_We_sat['Bike Id'].value_counts()[:30]
sWEsun = dF_We_sun['Bike Id'].value_counts(ascending=True)[:30]
sWEsat, sWEsun

Les méthodes makeGraphTopBikes() et makeGraphLowBikes() prennent en paramètre un dataframe et un nombre de vélos qui est le nombre retenu pour étude de vélos de plus forte utilisation (resp. de moins forte utilisation).
Chaque graphe est construit par ses arrêtes, données par les points de départ et d'arrivée successifs des courses rapidement enchaînées par le vélo (ou respectivement les itinéraires de vélos les ayant menés à des lieux de peu d'utilisation). 

In [ ]:
#On affiche le graph (basic)
# Remarques:
# Pas besoin de graphe acyclique, ni de point ici : juste indentifier l'endroit où ils circulent :
nx.draw(G_sat, with_labels=True)

In [ ]:
#On affiche le graph (basic)
# Remarques:
# Pas besoin de graphe acyclique, ni de point ici : juste indentifier l'endroit où ils circulent :
nx.draw(G_sun, with_labels=True)

On voit ci-dessus que networkX permet de définir des graphes, dirigés ou non selon le cas d'usage, mais que la représentation graphique finalement ne fait qu'en contrôler la cohérence.

Une représentation sur une carte nous a paru nécessaire pour interpréter les datas (retrouver des zones où le flux, la demande, est la plus forte (intuition du centre ville..), respectivement les zones de moindre demande (intuition de zones périphériques). Nous aurions certes pu explorer la possibilité d'adjoindre à chaque arrête la distance entre les noeuds..

Ci dessous donc l'affichage des graphes/dataframes sur la map, le code ayant consisté à faire une jointure entre les deux.

In [ ]:
m2 = Map(center=(51.50, -0.12), zoom=13,basemap=basemaps.Esri.WorldTopoMap)
m2.add_control(FullScreenControl())

#Prends le DF contenant les Coord Gp, prend un Graph (orienté ou non) et affiche tous les noeuds en marker (non clustered)
Single_Markers = Give_Marker(df_Coord_Gps_sat,G_sat)
Single_Markers_L = Give_Marker(df_Coord_Gps_sun,G_sun)


#Prends le DF contenant les Coord Gps, prend un graph (orienté) et affiche tous les edges en antpath
my_path=Prod_Ant_Path(df_Coord_Gps_sat,G_sat)
my_path_L=Prod_Ant_Path(df_Coord_Gps_sun,G_sun, "#f42069")

for i in my_path :
    m2.add_layer(my_path[i])
for j in my_path_L:
    m2.add_layer(my_path_L[j])
    
#for marker in Single_Markers :
#    m2.add_layer(marker)
    
#for marker_L in Single_Markers_L :
#    m2.add_layer(marker_L)

# EN VERT CI DESSOUS LES 10 VELOS LES PLUS UTILISES LE SAMEDI
# EN ROUGE CI DESSOUS LES 10 VELOS LES MOINS UTILISES LE DIMANCHE
display(m2)

In [ ]:
sSem = dF_semaine['Bike Id'].value_counts()[:30]
sSem

## B - Etude sur l'utilisation des vélos durant la semaine (Question 1)

On s'intéresse cette fois à tous les parcours rélisés en semaine aux horaires d'embauche et de débauche.

In [ ]:
#CREATION DES DATAFRAMES : param :  saturday ou sunday, Duration

#  vélos les plus utilisés en semaine EN VERT sur la map
dF_semaine =Get_Slice(df,"week", (7,0), (9,0))

# CREATION DES GRAPHES : param : nbre de vélos étudiés
G_semaine = makeGraphTopBikes(dF_semaine, 10)

# CREATION DES MARKERS (Pas forcément affichés pour lisibilité)
df_Coord_Gps_semaine = Prod_Coord_Df(dF_semaine)


Voyons l'ID des 10 vélos les plus utilisés en semaine :

In [ ]:
dF_semaine['Bike Id'].value_counts()[:30]


In [ ]:
#On affiche le graph (basic)
# Remarques:
# Pas besoin de graphe acyclique, ni de point ici : juste indentifier l'endroit où ils circulent :
# La question est : peut on retrouver le centre ville? une zone particuliere?
nx.draw(G_semaine, with_labels=True)

In [ ]:
m5 = Map(center=(51.50, -0.12), zoom=13,basemap=basemaps.Esri.WorldTopoMap)
m5.add_control(FullScreenControl())

#Prends le DF contenant les Coord Gp, prend un Graph (orienté ou non) et affiche tous les noeuds en marker (non clustered)
Single_Markers_sem = Give_Marker(df_Coord_Gps_semaine,G_semaine)

#Prends le DF contenant les Coord Gps, prend un graph (orienté) et affiche tous les edges en antpath
my_path_semaine=Prod_Ant_Path(df_Coord_Gps_semaine,G_semaine)

for i in my_path_semaine :
    m5.add_layer(my_path_semaine[i])
    
#for marker in Single_Markers_sem :
#    m2.add_layer(marker)
   
display(m5)

Comparaison des id de vélos sur-utilisés le samedi et à l'embauche le matin en semaine:

In [ ]:
len(set(sWEsat.index)),len(set(sWEsun.index)),len(set(sSem.index))
set(sWEsat.index).isdisjoint( set(sSem.index))


## Conclusion à la question 1

Nous avons donc:
- construit des sous-échantillons de notre dataframe complet identifiant (pour lisibilité sur les maps):
    - les 10 vélos les plus utilisés, le samedi (de minuit à 23h30)
    - les 10 vélos les moins utilisés, dimanche (de minuit à 23h30)
    - les 10 vélos les plus utilisés en semaine (de 7h à 9h)
- affichés les parcours correspondants.

En conclusion à la question 1, nous dirons : 

- qu'après variation de tous les paramètres, nous n'avons jamais pu afficher une carte qui présenterait des parcours de vélos (parmi les plus utilisés comme parmi les moins utilisés) vraiment centrés sur une zone : en fait, le bruit est tel que tous les parcours vus sont globalement isotropes ! Les vélos ne demeurent jamais dans la même zone géographique.. 

- que l'affichage ne permet finalement jamais de trouver une explication géographique évidente au sur ou sous-usage affiché : les vélos ne restent pas dans une zone particulière dans les deux cas. Aussi n'avons nous pas jugé utile d'afficher une carte des 150 parcourcours d'outliers (utilisés plus de 22 fois), puisque les 10 premiers seuls témoignent d'une très forte non localisation,

- l'identification des numéros de vélos sur-utilisés le samedi et en semaine montre que ces deux ensembles sont différents, ce qui montre qu'un vélo très utilisé le samedi ne l'est pas en semaine et réciproquement. On peut donc penser d'après cela que l'aléa prévaut sur l'idée que certains vélos ont des caractéristiques différentes, plus attrayantes.

## C - Visualisation d'une localité (Question 2)

Nous affichons ci-dessous les parcours des vélos loués à 8h (en vert) et à 17h (en rouge), afin de d'illustrer encore une recherche de localité.
Comme la réponse à la question 1 le laisse penser, ce type d'affichage montre une distribution géographique très isotrope, et pas une sur-représentation du centre ville par exemple.

In [ ]:
# tous vélos utilisés de 8 à 9h, EN VERT sur la map
dF_semaine_matin =Get_Slice(df,"week", (8,0), (8,1))
# CREATION DES GRAPHES : param : nbre de vélos étudiés
G_sem_matin = makeGraphTopBikes(dF_semaine_matin, len(dF_semaine_matin))

# tous vélos utilisés de 1730 à 1830h, EN ROUGE sur la map
dF_semaine_soir =Get_Slice(df,"week", (17,30), (17,31))
# CREATION DES GRAPHES : param : nbre de vélos étudiés
G_sem_soir = makeGraphTopBikes(dF_semaine_soir, len(dF_semaine_soir))

# Création du dataframe des coordonnées GPS des stations concernées:
df_Coord_Gps_matin = Prod_Coord_Df(dF_semaine_matin)
df_Coord_Gps_soir = Prod_Coord_Df(dF_semaine_soir)

#Affichage de la map
m3 = Map(center=(51.50, -0.12), zoom=13,basemap=basemaps.Esri.WorldTopoMap)
m3.add_control(FullScreenControl())

#Affichage en chemin de fourmi sur map (sans les markers pour la lisibilité)
#(Prend le DF contenant les Coord Gps, prend un graph (orienté) et affiche tous les edges en antpath)
my_path_matin = Prod_Ant_Path(df_Coord_Gps_matin,G_sem_matin)
my_path_soir = Prod_Ant_Path(df_Coord_Gps_soir,G_sem_soir, "#f42069")

#Cidessou on n'affiche que la matin, trajets démarrés entre 8h et 8h 01 pour lisibilité
for i in my_path_matin :
    m3.add_layer(my_path_matin[i])
for j in my_path_soir:
    m3.add_layer(my_path_soir[j])

Single_Markers = Give_Marker(df_Coord_Gps_matin,G_sem_matin)

#for marker in Single_Markers :
#    m3.add_layer(marker)

#PLUSIEURS MINUTES

In [ ]:
display(m3)

Ci-dessous la représentation non spatiale de networkX : 

In [ ]:
nx.draw(G_sem_matin, with_labels=True)

## Conclusion à la question 2

Malgré la possibilité de filtrer les données de notre dataframe selon de nombreux paramètres (semaine/weekend, tranche horaire, fréquence d'utilisation des vélos), la représentation sur carte n'a jamais fait apparître de localité : les vélos ne restent jamais, par hasard, dans une même zone.
